In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import datetime

%config IPCompleter.greedy=True

In [2]:
xl = pd.ExcelFile('/Users/sergey/Documents/TUM/IDP/prepared_instruments.xlsx')

In [3]:
dfTMP = xl.parse("TS", skiprows = 0, skip_footer =False)

In [39]:
def absPortf_Risk_of_loss(listofInv, numDays= 2, MoneyVol = 1000, startDt = '2016-11-25', endDt ='2016-11-30' ):
    """
    Measures the percentage of outcomes below a certain totalreturn level, usually 0%. 
    """
    
    #print(listofInv.head(9))
    colListFull  = list(listofInv.columns.values)

    #listRisk_99 = []
    
    startDate = np.datetime64( startDt)
    endDate= np.datetime64( endDt)
    
    ##remove Timestamp, Trade Open, Trade Close
    listofInv=pd.DataFrame(listofInv.ix[1:,0:])

    listofInv=pd.DataFrame(listofInv)
    #listofInv['Date'] = [time.date() for time in listofInv['Date']]

    listofInv['Date'] =  pd.to_datetime(listofInv['Date'], format='%Y-%m-%d')
    listofInv['Date'] = pd.to_datetime(listofInv['Date'])  

    mask = (listofInv['Date'] >= startDate) & (listofInv['Date'] <= endDate)
    #print (mask)
    listofInv = listofInv.loc[mask]  
    #print(listofInv.head(12))
    
    del colListFull[0]

    #drop column from the Data frame
    listofInv = listofInv.drop('Date', 1)
    
    colListPrime  = list(listofInv.columns.values)
    colListPrime = colListPrime[::2] 
    print(colListPrime)
    
    set1=pd.DataFrame()

    colListPrime = list(listofInv.columns.values)

    #print(listofInv.head(10))
    #print(colListFull)
    
    '''Test for handling NaNs.
    ##listofInv.loc[listofInv['TGTG.DE.1'] == 0.68, 'TGTG.DE.1'] = np.nan
        result: if at least one cell of a row contains NaN, then the whole row is discarded.
    '''
    
    print("Posrf:")
    
    print(listofInv)
    
    for i in range(0, len(colListFull), 2):
        listofInv = listofInv[pd.notnull(listofInv[listofInv.columns[(i)]])]
        listofInv = listofInv[pd.notnull(listofInv[listofInv.columns[(i+1)]])]
        
        set2 =(listofInv[listofInv.columns[i+1]].astype(float)) - (listofInv[listofInv.columns[i]].astype(float))
        set1 = pd.concat([set1, set2] , axis=1)
        del set2

    #set1.columns = [colListPrime]
    #print(set1.head())


    SumSet = set1.groupby((np.arange(len(set1.columns)) //len(set1.columns)*10 ) + 1, axis=1).sum().add_prefix('sum')
    ##SumSet["rets"] = SumSet["sum1"].pct_change()
    
    SumSet.replace(np.NaN,0 , inplace =True) 
    SumSet = SumSet.replace([np.inf, -np.inf], 0)
    SumSet = SumSet.astype(float)
    
    
    #setPortfVaR = pd.DataFrame()

    print(SumSet)
    ##print(((SumSet < 0).sum(1)).sum())
    
    Risk_of_loss = round(((SumSet < 0).sum(1)).sum()/SumSet.shape[0]*100, 2)
    print("Risk of Loss is: " + str(Risk_of_loss) + "%")
    
#dfTMP = xl.parse("TS", skiprows = 0, skip_footer =False)
##call for 4 Stocks:
absPortf_Risk_of_loss(pd.DataFrame(dfTMP.ix[1:,0:5]), 21, 1000, '2016-11-20','2016-11-30' )


['TGTG.DE', 'UUUG.DE']
Posrf:
  TGTG.DE TGTG.DE.1 UUUG.DE UUUG.DE.1
2    0.63      0.68   0.641     0.641
3    0.68     0.696    0.65      0.66
4   0.686       0.7    0.65      0.65
5   0.702       0.7     NaN       NaN
6     NaN       NaN   0.661     0.656
7   0.718     0.718   0.664     0.664
8   0.796     0.728    0.65     0.647
9     0.8     0.752   0.646     0.645
    sum1
2  0.050
3  0.026
4  0.014
5 -0.002
7  0.000
8 -0.071
9 -0.049
Risk of Loss is: 42.86%


In [ ]:
def ADJUSTED_FOR_RUNNING_FROM_MAIN_absPortf_Risk_of_loss(Inv, numDays= 2, MoneyVol = 1000, startDt = '2016-11-25', endDt ='2016-11-30' ):
    """
    Measures the percentage of outcomes below a certain totalreturn level, usually 0%. 
    """
    
    #print(listofInv.head(9))
    
    colListPrime = FormColumns(Inv)
    
    set1=pd.DataFrame()
    #print(colListPrime)
    #listStdDev = []

    listofInv = FormDataforGivenDates(Inv,startDt,endDt)

    
    ##drop column from the Data frame
    listofInv = listofInv.drop('Date', 1)  
    colListFull  = list(listofInv.columns.values)
    
    print(colListFull)
    print(listofInv)
    
    for i in range(0, len(colListFull), 2):
        listofInv = listofInv[pd.notnull(listofInv[listofInv.columns[(i)]])]
        listofInv = listofInv[pd.notnull(listofInv[listofInv.columns[(i+1)]])]
        
        set2 =(listofInv[listofInv.columns[i+1]].astype(float)) - (listofInv[listofInv.columns[i]].astype(float))
        set1 = pd.concat([set1, set2] , axis=1)
        del set2

    #set1.columns = [colListPrime]
    #print(set1.head())

    SumSet = set1.groupby((np.arange(len(set1.columns)) //len(set1.columns)*10 ) + 1, axis=1).sum().add_prefix('sum')
    ##SumSet["rets"] = SumSet["sum1"].pct_change()
    
    SumSet.replace(np.NaN,0 , inplace =True) 
    SumSet = SumSet.replace([np.inf, -np.inf], 0)
    SumSet = SumSet.astype(float)
    
    
    #setPortfVaR = pd.DataFrame()

    print(SumSet)
    ##print(((SumSet < 0).sum(1)).sum())
    
    Risk_of_loss = round(((SumSet < 0).sum(1)).sum()/SumSet.shape[0]*100, 2)
    print("Risk of Loss is: " + str(Risk_of_loss) + "%")
    
#dfTMP = xl.parse("TS", skiprows = 0, skip_footer =False)
##call for 4 Stocks:
ADJUSTED_FOR_RUNNING_FROM_MAIN_absPortf_Risk_of_loss(pd.DataFrame(dfTMP.ix[1:,0:5]), 21, 1000, '2016-11-20','2016-11-30' )
